In [12]:
fm = 1
hbarc = 0.197327053
c = 1  # speed of light
hbar = 1  # reduced Planck constant

GeV = 1 / hbarc  # Giga-electronvolt
MeV = 1e-3 * GeV  # Mega-electronvolt

g = 5.625e26 * MeV  # gram
kg = 1e3 * g  # kilogram
cm = 1e13 * fm  # centimeter
m = 100 * cm  # meter
km = 1e5 * cm  # kilometer
s = 3e10 * cm  # second

dyn = g * cm / s**2  # dyne
dyn_cm_2 = dyn / cm**2  # dyne / cm^2
g_cm_3 = g / cm**3  # gram / cm^3
erg = dyn * cm  # ἐργον energy unit

m_n = 939.565 * MeV  # mass of neutron
n0 = 0.16 / fm**3  # saturation density

e0 = m_n * n0  # saturation energy density
G = 6.6743e-8 * dyn * cm**2 / g**2  # gravitational constant
Msun = 1.989e33 * g  # mass of sun

In [13]:
import numpy as np
from scipy.interpolate import interp1d

# from TOVsolver import unit
from TOVsolver.main import OutputMR

def maxium_central_density(energy_density, pressure, central_densitys=np.logspace(14.3, 15.6, 5) * g_cm_3, num2=30):
    """Outputs the maxium central density of a stable EoS
    Args:
        energy_density (numpy 1Darray): Density of EoS
        pressure (numpy 1Darray): Pressure of EoS
        central_densitys (numpy 1Darray): The range of central density
        num2 (int): The number of segments in the density interval of second search. At least 5 points
    Returns:
        density (float): The maxium central density, in unit.g_cm_3
    """
    ############## Below is the main part of two searches for peaks
    ######## First search
    Ms = [-1,-2] # Meaningless initialization, only to ensure that the following 'if (i>0) and (Ms [-1]<=Ms [-2])' statements can run properly
    store_d_range = [central_densitys[-2], central_densitys[-1]] # When the following loop does not output a result, initialization here will have its meaning
    # Find the extremum point within the predetermined range of central density and return the central density near the extremum point
    for i, rho in enumerate(central_densitys):
        M, R = OutputMR('', energy_density, pressure, [rho])[0]
        Ms.append(M)
        if (i>0) and (Ms[-1] <= Ms[-2]):
            store_d_range = [central_densitys[i-2], central_densitys[i]] 
            break
    Ms_larg = Ms[-1] # Used to store the mass corresponding to the maximum density during the first peak search
    
    ######## Second search
    Ms = [-1,-2] # Reinitialize
    # Update and refine the central density range, and ultimately return the central density of the extremum points
    store_d_range = np.geomspace(store_d_range[0], store_d_range[1], num2) # At least 5 points
    # Note that the first and last points have already been calculated, so there is no need to traverse them
    for i, rho in enumerate(store_d_range[1:-1]):
        M, R = OutputMR('', energy_density, pressure, [rho])[0]
        Ms.append(M)
        if Ms[-1] <= Ms[-2]:    # Note that due to the second peak search refining the interval, the result is generally not obtained at the first point.
                                # Therefore, initializing Ms=[-1, -2] is acceptable
            return store_d_range[1:][i-1]
    # When the above peak search fails, continue comparing the last point
    if Ms_larg < Ms[-1]:
        return store_d_range[-2]
    else:
        return store_d_range[-1]

In [14]:
from scipy.constants import pi
from scipy.integrate import ode
from scipy.interpolate import interp1d


In [15]:
def TOV(r, y, inveos):
    pres, m = y

    # eps = 10**inveos(np.log10(pres))
    eps = inveos(pres)
    dpdr = -(eps + pres) * (m + 4.0 * pi * r**3.0 * pres)
    dpdr = dpdr / (r * (r - 2.0 * m))
    dmdr = 4.0 * pi * r**2.0 * eps

    return np.array([dpdr, dmdr])

In [16]:
def solveTOV(center_rho, Pmin, eos, inveos):
    """Solve TOV equation from given Equation of state in the neutron star 
    core density range

    Args:
        center_rho(array): This is the energy density here is fixed in main
        that is range [10**14.3, 10**15.6] * unit.g_cm_3
        
        Pmin (float): In unit.G / unit.c**4
        
        eos (function): pressure vs. energy density, energy density in unit.G / unit.c**2, pressure in unit.G / unit.c**4
        
        inveos (function): energy density vs. pressure

    Returns:
        Mass (float): The Stars' masses
        Radius (float): The Stars's radius
    """
    r = 4.441e-16 * cm
    dr = 10.0 * cm

    pcent = eos(center_rho)
    P0 = (
        pcent
        - (4.0 * pi / 3.0) * (pcent + center_rho) * (3.0 * pcent + center_rho) * r**2.0
    )
    m0 = 4.0 / 3.0 * pi * center_rho * r**3.0
    stateTOV = np.array([P0, m0])

    sy = ode(TOV, None).set_integrator("dopri5")

    # have been modified from Irida to this integrator
    sy.set_initial_value(stateTOV, r).set_f_params(inveos)

    while sy.successful() and stateTOV[0] > Pmin:
        stateTOV = sy.integrate(sy.t + dr)
        dpdr, dmdr = TOV(sy.t + dr, stateTOV, inveos)
        dr = 0.46 * (1.0 / stateTOV[1] * dmdr - 1.0 / stateTOV[0] * dpdr) ** (-1.0)

    # at the end of this function, we rescale the quantities back
    return stateTOV[1] * c**2 / G, sy.t

In [17]:
import numpy as np
from scipy.optimize import root

# from TOVsolver.unit import g_cm_3, dyn_cm_2

def compute_EOS(rhos, theta,
              rho_t_start=4.3721E11*g_cm_3, P_t_start=7.7582E29*dyn_cm_2):
    """
    Calculate the pressure of a neutron star based on density using a piecewise polytropic equation of state (EOS).

    This function computes the pressure (`pres`) as a function of density (`rho`) by applying different polytropic indices
    (`gamma_1`, `gamma_2`, ..., `gamma_n`) within specified density thresholds (`rho_t_start`, `rho_t_1`, `rho_t_2`, ..., `rho_t_n-1`).
    The EOS is defined in n distinct regions:
    
    - **Low-density region:** `rho_t_start < rho <= rho_t_1`
    - **Intermediate-density region:** `rho_t_1 < rho <= rho_t_2`, ..., `rho_t_k < rho <= rho_t_k+1`, ..., `rho_t_n-2 < rho <= rho_t_n-1`
    - **High-density region:** `rho > rho_t_n-1`

    Parameters
    ----------
    rho : array-like
        An array of density values (in cgs units) at which to calculate the pressure.
    
    theta : array-like, length `2 * n - 1`
        A list or tuple containing the EOS parameters in the following order:
        
        - `gamma_1` (float): Polytropic index for the low-density region.
        - `gamma_2` to `gamma_2_n-1` (float): Polytropic index for the intermediate-density region.
        - `gamma_n` (float): Polytropic index for the high-density region.
        - `rho_t_1` (float): Density threshold between the low and intermediate-density regions (in cgs units).
        - `rho_t_2` to `rho_t_n-2` (float): Density for the intermediate regions (in cgs units).
        - `rho_t_n-1` (float): Density threshold between the intermediate and high-density regions (in cgs units).
    
    rho_t_start : float
        Start point of the density for polytropic EOS

    P_t_start : float
        Start point of the pressure for polytropic EOS
    Returns
    -------
    pres : ndarray
        An array of pressure values (in cgs units) corresponding to the input density values.
    """
    
    n = len(theta) // 2 + 1
    gammas = theta[:n]
    rho_ts = theta[n:]
    
    P_ts, ks = np.zeros((2, n))
    rho_ts = np.insert(rho_ts, 0, rho_t_start)

    # Calculate the values of P_t and k, based on the parameter gamma and rho_t
    for i in range(len(ks)):
        if i == 0:
            P_ts[i] = P_t_start
            ks[i] = P_ts[i] / (rho_ts[i]**gammas[i])
        else:
            P_ts[i] = ks[i-1] * (rho_ts[i]**gammas[i-1])
            ks[i] = P_ts[i] / (rho_ts[i]**gammas[i])

    # Construct judgement criteria for each section
    conds = [np.array(rhos > rho_ts[i]) & np.array(rhos <= rho_ts[i+1]) for i in range(n-1)]
    conds.append(rhos > rho_ts[-1])
    
    # Build polytrope functions to compute the pressure for each section
    functions = [lambda rho, k=ks[i], g=gammas[i]: k * (rho ** g) for i in range(n)]
    
    # Calculate pressure by using np.piecewise, based on the conds and functions we defined above.
    pres = np.piecewise(rhos, conds, functions)

    return pres

def fun_gamma_max(rho2, rho1, p1):
    """Outputs the maximum gamma for given densities and pressure at both ends of a section of polytropic function.
    Args:
        rho2 (float): density at the end point.
        rho1 (float): density at the start point.
        p1 (float): pressure at the start point.
    Returns:
        gamma_max (float): the maximum gamma.
    """
    def fun(x):
        a1 = np.log(rho2/p1/x)
        a2 = np.log(rho2/rho1)
        return a1 / a2 - x

    gamma_max = root(fun, [3/4]).x[0]
    return gamma_max

In [18]:
#Tolos_crust_out = np.loadtxt('Tolos_crust_out.txt', delimiter='  ')
Tolos_crust_out = np.loadtxt('/Users/DELL/CompactOject/Test_Case/Tolos_crust_out.txt', delimiter=None, comments='#', usecols=(0, 1, 2, 3, 4))
eps_crust = Tolos_crust_out[:,3] * g_cm_3
pres_crust = Tolos_crust_out[:,4] * dyn_cm_2

# rho_crust_end and P_crust_end are the end of outer crust EoS and the start point of polytrope EoS
rho_crust_end = eps_crust[-1]
P_crust_end = pres_crust[-1]

# eps_core is the density array from inner crust to core, is the palce where we want to establish the polytrope EoS
# cent_densitys is the central density range of a neutron star
eps_core = np.logspace(11.7, 15.6, 1000, base=10) * g_cm_3
cent_densitys = np.logspace(14.3, 15.6, 30) * g_cm_3

In [19]:
import InferenceWorkflow.prior as prior

In [20]:
parameters = ['rho1', 'rho2', 'rho3', 'gamma1', 'gamma2', 'gamma3', 'gamma4', 'd1', 'd2', 'd3']
derived_param_names = ['d_max']

gamma_lo_ranges = [1, 0.1, 0.1, 0.1]
gamma_up_ranges = [1.5, 4, 8, 4]
def prior_transform(cube):
    para = cube.copy()

    # Output appropriate gamma
    # When we randomly scatter a gamma at one segement, its maxium value is constrained by the start point's pressure, density,
    # and the end point's density. Therefore in each segement, we need call 'fun_gamma_max' to get a sutiable range of gamma.
    para[0] = prior.flat_prior(0.04/0.16*e0, 0.065/0.16*e0, cube[0])
    para[1] = prior.flat_prior(1.5*e0, 8.3*e0, cube[1])
    para[2] = prior.flat_prior(para[1], 8.3*e0, cube[2])
    rho_s = [para[0], para[1], para[2], eps_core[-1]]
    gamma_s = []
    k = 0
    pt = 0
    for i in range(4):
        if i==0:
            gamma_max = fun_gamma_max(rho_s[i], rho_crust_end, P_crust_end)
            if gamma_up_ranges[i] < gamma_max:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_up_ranges[i], cube[3+i])
            else:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_max, cube[3+i])
            gamma_s.append(para[3+i])
            k = P_crust_end / (rho_crust_end**gamma_s[-1])
            pt = k*rho_s[0]**gamma_s[-1]
        else:
            gamma_max = fun_gamma_max(rho_s[i], rho_s[i-1], pt)
            if gamma_up_ranges[i] < gamma_max:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_up_ranges[i], cube[3+i])
            else:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_max, cube[3+i])
            gamma_s.append(para[3+i])
            k = pt / (rho_s[i-1]**gamma_s[-1])
            pt = k*rho_s[i]**gamma_s[-1]

    gammas = np.array([para[3],para[4],para[5],para[6]])
    rho_ts= np.array([para[0],para[1],para[2]])
    theta = np.append(gammas, rho_ts)
    pres = compute_EOS(eps_core, theta)
    eps = np.hstack((eps_crust, eps_core))
    pres = np.hstack((pres_crust, pres))
    
    d_max = maxium_central_density(eps, pres, cent_densitys)
    para[7:] = 14.3 + (np.log10(d_max / g_cm_3) - 14.3) * cube[7:]

    return np.append(para, d_max) # d_max is in natural unit, but d1,d2,d3 are the result of taking the logarithm of density in g_cm3

In [21]:
import csv
import sys
def file_read(input_file):
    """file_read

    Reads a csv file of denisty and pressure given by the user.

    Args:
        input_file (string): string. File to be opened and parsed.

    Returns:
        array: two 1Darray numpy arrays, one corresponding to density and one corresponding to pressrure.
    """

    data_list = []
    density_list = []
    pressure_list = []
    with open(input_file) as csvfile:
        file_read = csv.reader(csvfile, delimiter=",")
        data_list = [row for row in file_read]
    for row in data_list:
        density_list.append(float(row[0]))
        pressure_list.append(float(row[1]))

    # Make the lists numpy arrays
    density_array = np.array(density_list)
    pressure_array = np.array(pressure_list)

    return density_array, pressure_array


def EOS_check(density, pressure):
    """file_read

    Checks that the derivative (drho/dp) is positive.

    Args:
        density (array): numpy 1Darray. Density array to be checked.
        pressure (array): numpy 1Darray. Pressure array to be checked.

    Returns:
        array: two arrays, one corresponding to density and one corresponding to pressure or ends the function and prints
        invalid equation of state.
    """

    dp = np.diff(pressure)  # dy
    drho = np.diff(density)  # dx

    for value in drho:
        if value == 0:
            print("This is not a valid equation of state, 0")
            sys.exit()

    dpdrho = dp / drho  # dydx

    for value in dpdrho:
        if value < -1e-3:
            print(f"dpdrho = {value} is not a valid equation of state")
            sys.exit()

    return density, pressure

In [22]:
def EOS_import(file_name="", density=0, pressure=0):
    """EOS_import

    Imports density and pressure from csv or array, checks them, and returns them.

    Args:
        file_name (string, optional): string. CSV file to be opened.
        density (array, optional): numpy 1Darray. Passed into a check function and returned if valid.
        pressure (array, optional): numpy 1Darray. Passed into a check function and returned if valid.

    Returns:
        array: checked density and pressure.
    """

    if not file_name:
        density_checked, pressure_checked = EOS_check(density, pressure)
        return density_checked, pressure_checked

    input_file = file_name

    density, pressure = file_read(input_file)


    density_checked, pressure_checked = EOS_check(density, pressure)

    # tzzhou: migrating to new units
    density *= c**2 / G * g_cm_3
    pressure *= c**4 / G * dyn_cm_2


    return density, pressure

In [23]:
# Global Variables
def OutputMR(input_file="", density=[], pressure=[], central_density_range=np.logspace(14.3, 15.6, 50) * g_cm_3):
    """Outputs the mass, radius, and tidal deformability
    Args:
        file_name (string, optional): string. CSV file to be opened.
        density (array, optional): numpy 1Darray. Passed into a check function and returned if valid.
        pressure (array, optional): numpy 1Darray. Passed into a check function and returned if valid.

    Returns:
        MR (tuple): tuple with mass, radius.
    """

    #############This is something we need to change, like the input for this EOS import should
    ############# be one file contatining Whole EOS. that first column is density and second is pressure
    energy_density, pressure = EOS_import(input_file, density, pressure)
    ############# Lets the user only input the EOS file path, then this EOS_import should have file
    ############# as input. and the outputMR should have a file as input too?

    # Notice that we only rescale quantities inside this function
    energy_density = energy_density * G / c**2
    pressure = pressure * G / c**4

    # eos = UnivariateSpline(np.log10(energy_density), np.log10(pres), k=1, s=0)
    # inveos = UnivariateSpline(np.log10(pres), np.log10(energy_density), k=1, s=0)
    # We could change this to Double Log Interpolation。

    unique_pressure_indices = np.unique(pressure, return_index=True)[1]
    unique_pressure = pressure[np.sort(unique_pressure_indices)]

    # Interpolate pressure vs. energy density
    eos = interp1d(energy_density, pressure, kind="cubic", fill_value="extrapolate")

    # Interpolate energy density vs. pressure
    inveos = interp1d(
        unique_pressure,
        energy_density[unique_pressure_indices],
        kind="cubic",
        fill_value="extrapolate",
    )

    Pmin = pressure[20]

    Radius = []
    Mass = []

    # This following step is to make a dicision whether the EOS ingredients is always increase. We can do that outsie of this main to the
    # EOS import.
    # if   all(x<y for x, y in zip(eps_total_poly[:], eps_total_poly[[1:])) and all(x<y for x, y in zip(pres_total_poly[j][:], pres_total_poly[j][1:])):
    for i in range(len(central_density_range)):
        try:
            center_rho = central_density_range[i] * G / c**2
            M, R = solveTOV(center_rho, Pmin, eos, inveos)
            Mass.append(M)
            Radius.append(R)
        # This is sentense is for avoiding the outflow of the result, like when solveTOV blow up because of ill EOS, we need to stop
        except OverflowError as e:
            # print("This EOS is ill-defined to reach an infinity result, that is not phyiscal, No Mass radius will be generated.")
            break
    MR = np.vstack((Mass, Radius)).T
    # print("Mass Radius file will be generated and stored as  2-d array. The first column is mass, second one is Radius")

    return MR

In [ ]:
import pandas as pd
from scipy.interpolate import interp1d

datas = [
    [10, 1.97, 0.5, 0.0985],
    [13, 1.44, 0.65, 0.072],
    [12, 2.07, 0.6, 0.1035]
]
datas = pd.DataFrame(datas, index=[1,2,3], columns=['R', 'M', 'Rsigma', 'Msigma'])


Rvalue = datas['R'].values * km
Mvalue = datas['M'].values * Msun

R_sigma = datas['Rsigma'].values * km
M_sigma = datas['Msigma'].values * Msun

f1 = np.log(1/(R_sigma*M_sigma*(np.sqrt(2*np.pi))**2)).sum()


def MRlikihood_Gaussian(eps_total, pres_total, x, d1):
    """
    Computing likelihood from a simulation Gaussian distribution of MR measurement.
    
    Args:
        eps_total (array): The energy density of full EoS in MeV/fm3, times a G/c**2 factor.
        pres_total (array): The pressure from full EoS model in MeV/fm3, times a G/c**4 factor.
        x (list or array): [Mvalue, Rvalue, Mwidth, Rwidth], each should be array-like with the same length.
        d1 (array): The sampled density (log10) for each measurement.
    
    Returns:
        likelihood (float): Sum of log likelihoods for all measurements.
    """
    Mvalue, Rvalue, Mwidth, Rwidth = x  # Each should be array-like with the same length
    N = len(Mvalue)
    likelihood = 0.0
    for i in range(N):
        if d1[i] == 0:
            likelihood += -1e101
            print(f"MRlikihood_Gaussian: d1[{i}] is 0, assigning -1e101")
            continue
        d1_val = 10 ** d1[i]  # Convert log density to linear density
        # Check if epsilon and pressure are monotonically increasing
        if not (np.all(np.diff(eps_total) > 0) and np.all(np.diff(pres_total) > 0)):
            likelihood += -1e101
            print(f"MRlikihood_Gaussian: Epsilon or pressure not monotonically increasing for d1[{i}]")
            continue
        # Compute MR
        MR = OutputMR("", eps_total, pres_total, [d1_val * g_cm_3])[0]
        if len(MR) == 0:
            likelihood += -1e101
            print(f"MRlikihood_Gaussian: MR is empty for d1[{i}]")
            continue
        # Ensure MR has at least two elements: [Mass, Radius]
        if len(MR) < 2:
            likelihood += -1e101
            print(f"MRlikihood_Gaussian: MR has insufficient elements for d1[{i}]")
            continue
        # Extract Mass and Radius
        mass = MR[0]  # in Msun units
        radius = MR[1]  # in km units
        # Compute likelihood fx for this data point
        fx = (1 / (Rwidth[i] * Mwidth[i] * (np.sqrt(2 * np.pi)) ** 2)) * np.exp(
            -((radius - Rvalue[i]) ** 2) / (2 * (Rwidth[i] ** 2)) -
            ((mass - Mvalue[i]) ** 2) / (2 * (Mwidth[i] ** 2))
        )
        # Avoid log(0)
        if fx <= 0:
            likelihood += -1e101
            print(f"MRlikihood_Gaussian: fx <= 0 for d1[{i}]")
            continue
        likelihood += np.log(fx)
    return likelihood

def log_prior(theta):
    """
    Computes the log prior for the given parameters.
    
    Args:
        theta: Array of parameters [rho1, rho2, rho3, gamma1, gamma2, gamma3, gamma4, d1, d2, d3].
    
    Returns:
        Scalar log prior value.
    """
    try:
        rho1, rho2, rho3, gamma1, gamma2, gamma3, gamma4, d1, d2, d3 = theta
        # Check rho1, rho2, rho3 bounds
        if not (0.04 / 0.16 * e0 <= rho1 <= 0.065 / 0.16 * e0):
            print(f"log_prior: rho1={rho1} out of bounds")
            return -1e101
        if not (1.5 * e0 <= rho2 <= 8.3 * e0):
            print(f"log_prior: rho2={rho2} out of bounds")
            return -1e101
        if not (rho2 <= rho3 <= 8.3 * e0):
            print(f"log_prior: rho3={rho3} out of bounds or not >= rho2")
            return -1e101
        # Check gamma bounds
        rho_s = [rho1, rho2, rho3, eps_core[-1]]
        gamma_s = [gamma1, gamma2, gamma3, gamma4]
        k = 0
        pt = 0
        for i in range(4):
            if i == 0:
                gamma_max = fun_gamma_max(rho_s[i], rho_crust_end, P_crust_end)
                if gamma_up_ranges[i] < gamma_max:
                    if not (gamma_lo_ranges[i] <= gamma_s[i] <= gamma_up_ranges[i]):
                        print(f"log_prior: gamma{i+1}={gamma_s[i]} out of bounds")
                        return -1e101
                else:
                    if not (gamma_lo_ranges[i] <= gamma_s[i] <= gamma_max):
                        print(f"log_prior: gamma{i+1}={gamma_s[i]} out of bounds")
                        return -1e101
                k = P_crust_end / (rho_crust_end ** gamma_s[i])
                pt = k * rho_s[0] ** gamma_s[i]
            else:
                gamma_max = fun_gamma_max(rho_s[i], rho_s[i - 1], pt)
                if gamma_up_ranges[i] < gamma_max:
                    if not (gamma_lo_ranges[i] <= gamma_s[i] <= gamma_up_ranges[i]):
                        print(f"log_prior: gamma{i+1}={gamma_s[i]} out of bounds")
                        return -1e101
                else:
                    if not (gamma_lo_ranges[i] <= gamma_s[i] <= gamma_max):
                        print(f"log_prior: gamma{i+1}={gamma_s[i]} out of bounds")
                        return -1e101
                k = pt / (rho_s[i - 1] ** gamma_s[i])
                pt = k * rho_s[i] ** gamma_s[i]
        # Compute EOS
        theta_eos = np.append(gamma_s, [rho1, rho2, rho3])
        pres = compute_EOS(eps_core, theta_eos)
        eps = np.hstack((eps_crust, eps_core))
        pres_full = np.hstack((pres_crust, pres))
        # Compute d_max
        d_max = maxium_central_density(eps, pres_full, cent_densitys)
        # Compute d1, d2, d3
        d1_val = 14.3 + (np.log10(d_max / g_cm_3) - 14.3) * d1
        d2_val = 14.3 + (np.log10(d_max / g_cm_3) - 14.3) * d2
        d3_val = 14.3 + (np.log10(d_max / g_cm_3) - 14.3) * d3
        # Check d1_val, d2_val, d3_val bounds
        if not (14.3 <= d1_val <= np.log10(d_max / g_cm_3)):
            print(f"log_prior: d1_val={d1_val} out of bounds")
            return -1e101
        if not (14.3 <= d2_val <= np.log10(d_max / g_cm_3)):
            print(f"log_prior: d2_val={d2_val} out of bounds")
            return -1e101
        if not (14.3 <= d3_val <= np.log10(d_max / g_cm_3)):
            print(f"log_prior: d3_val={d3_val} out of bounds")
            return -1e101
        # All checks passed
        return 0.0
    except Exception as e:
        print(f"log_prior: Exception occurred: {e}")
        return -1e101

def log_likelihood(theta):
    """
    Computes the log likelihood for the given parameters.
    
    Args:
        theta: Array of parameters [rho1, rho2, rho3, gamma1, gamma2, gamma3, gamma4, d1, d2, d3].
    
    Returns:
        Scalar log likelihood value.
    """
    # # Define x as [Mvalue, Rvalue, Mwidth, Rwidth] arrays
    # x = [Mvalue, Rvalue, M_sigma, R_sigma]
    # Extract d1, d2, d3
    d1, d2, d3 = theta[7], theta[8], theta[9]
    d_s = [d1, d2, d3]
    d_s_array = np.array(d_s)
    # Call the likelihood function
    likelihood = MRlikihood_Gaussian(eps_crust, pres_crust, x, d_s_array)
    return likelihood

def log_probability(theta):
    """
    Computes the combined log probability (log_prior + log_likelihood).
    
    Args:
        theta: Array of parameters [rho1, rho2, rho3, gamma1, gamma2, gamma3, gamma4, d1, d2, d3].
    
    Returns:
        Scalar log probability value.
    """
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -1e101

    ll = log_likelihood(theta)
    if not np.isfinite(ll):
        return -1e101

    return lp + ll

# def sample_initial_positions(nwalkers, ndim):
#     """
#     Samples initial positions for the walkers within the prior bounds.
    
#     Args:
#         nwalkers: Number of walkers.
#         ndim: Number of dimensions.
    
#     Returns:
#         Array of initial positions.
#     """
#     positions = []
#     attempts = 0  # To prevent infinite loops
#     max_attempts = nwalkers * 100

#     while len(positions) < nwalkers and attempts < max_attempts:
#         cube = np.random.uniform(0, 1, ndim)
#         params = prior_transform(cube)
#         theta = params  # Since prior_transform returns the 10 parameters
#         if log_prior(theta) > -1e101:
#             positions.append(theta)
#         attempts += 1

#     if len(positions) < nwalkers:
#         raise RuntimeError("Failed to initialize all walkers within the prior bounds.")

#     return np.array(positions)

In [ ]:
import emcee
# Number of dimensions and walkers
ndim = 9  # Number of parameters: rho1, rho2, rho3, gamma1, gamma2, gamma3, gamma4, d1, d2, d3
nwalkers = 1  # Number of walkers

# Initial positions: small Gaussian ball around the prior median
# Here, we'll sample uniformly within the prior ranges
initial_pos = []
for i in range(nwalkers):
    # Random values between 0 and 1 for each parameter
    cube = np.random.rand(ndim)
    para = prior_transform(cube)
    # Check if the initial position is valid (log_prior != -1e101)
    if log_prior(para) > -1e101:
        initial_pos.append(para)
    else:
        # If invalid, resample
        while True:
            cube = np.random.rand(ndim)
            para = prior_transform(cube)
            if log_prior(para) > -1e101:
                initial_pos.append(para)
                break
    if len(initial_pos) == nwalkers:
        break

initial_pos = np.array(initial_pos)

# Set up the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability)

# Run MCMC for 50 steps
print("Running MCMC...")
sampler.run_mcmc(initial_pos, 1, progress=True)
print("MCMC completed.")

# Access the samples
samples = sampler.get_chain(flat=True)
print(f"Number of samples: {samples.shape}")

# Save the samples to a file
np.savetxt("mcmc_samples.txt", samples)

# Optional: Plot the corner plot (requires corner library)
try:
    import corner
    labels = parameters
    fig = corner.corner(samples, labels=labels, truths=None, show_titles=True)
    fig.savefig("corner_plot.png")
    print("Corner plot saved as 'corner_plot.png'.")    
except ImportError:
    print("corner library not installed. Skipping corner plot.")
